In [1]:
class Parent:
    def __init__(self, amount):
        self.__amount = amount
        
    @property
    def amount(self):
        return self.__amount
    
    @amount.setter
    def amount(self, value):
        self.__amount = value

In [12]:
parent = Parent(100)
print(parent.amount)

100


In [9]:
class Child():
    def __init__(self, 
                 parent: Parent,
                 ):
        self.parent = parent
    
    def consume(self, amount):
        self.parent.amount -= amount
    
    @property    
    def parent_amount(self):
        return self.parent.amount

In [13]:
childs = [Child(parent) for _ in range(10)]
for child in childs:
    child.consume(10)
    print(child.parent_amount, parent.amount)

90 90
80 80
70 70
60 60
50 50
40 40
30 30
20 20
10 10
0 0
